CEBRA es un método de machine learning que puede usarse para comprimir series temporales de una forma que revela estructuras ocultas en la variabilidad de los datos. Se destaca especialmente en datos conductuales y neuronales registrados de manera simultánea.

Decodificación a partir de un embedding de CEBRA

En este notebook mostramos cómo:

- Decodificar etiquetas a partir de un embedding de CEBRA.

- Evaluar y comparar el rendimiento de la decodificación en embeddings obtenidos bajo diferentes hipótesis.

Esto se basa en lo presentado en *Schneider, Lee, Mathis.*

[ref con datos](https://colab.research.google.com/github/AdaptiveMotorControlLab/CEBRA-demos/blob/main/Demo_hippocampus.ipynb)

In [ ]:
!pip install --pre 'cebra[datasets, integrations]'

In [ ]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib as jl
import cebra.datasets
from cebra import CEBRA

from matplotlib.collections import LineCollection
import pandas as pd

## cargar los datos

In [ ]:
## completar con un npy (por ejemplo)

## usamos CEBRA para encontrar el espacio de baja dimension de nuestros datos

In [ ]:
#2. Define the model
max_iterations = 5000
cebra_posdir3_model = CEBRA(model_architecture='offset10-model',
                        batch_size=512,
                        learning_rate=3e-4,
                        temperature=1,
                        output_dimension=3,
                        max_iterations=max_iterations,
                        distance='cosine',
                        conditional='time_delta',
                        device='cuda_if_available',
                        verbose=True,
                        time_offsets=10)

In [ ]:
import cebra.models
print(cebra.models.get_options('offset*'))


['offset10-model', 'offset10-model-mse', 'offset5-model', 'offset1-model-mse', 'offset1-model', 'offset1-model-v2', 'offset1-model-v3', 'offset1-model-v4', 'offset1-model-v5', 'offset40-model-4x-subsample', 'offset20-model-4x-subsample', 'offset4-model-2x-subsample', 'offset36-model', 'offset36-model-dropout', 'offset36-model-more-dropout']


In [ ]:
# 3. Split data and labels
from sklearn.model_selection import train_test_split
split_idx = int(0.8 * len(hippocampus_pos.neural)) #suggest: 5%-20% depending on your dataset size

train_data = hippocampus_pos.neural[:split_idx]
valid_data = hippocampus_pos.neural[split_idx:]

train_continuous_label = hippocampus_pos.continuous_index.numpy()[:split_idx]
valid_continuous_label = hippocampus_pos.continuous_index.numpy()[split_idx:]

In [ ]:
# entrenamos el modelo --> recomendacion usar GPU la coalb
cebra_posdir3_model.fit(train_data, train_continuous_label)

In [ ]:
# guardamos el modelo
# 5. Save the model
import os
import tempfile
from pathlib import Path
tmp_file = Path(tempfile.gettempdir(), 'cebra.pt')
cebra_posdir3_model.save(tmp_file)

In [ ]:
# 6. cargamos el modelo y calculamos el embedding
cebra_posdir3_model = cebra.CEBRA.load(tmp_file)
train_embedding = cebra_posdir3_model.transform(train_data)
valid_embedding = cebra_posdir3_model.transform(valid_data)


In [ ]:
# visualizacion
import cebra.integrations.plotly

fig = cebra.integrations.plotly.plot_embedding_interactive(train_embedding,
                                                           embedding_labels=train_continuous_label[:,0],
                                                           title = "CEBRA-Behavior Train",
                                                           markersize=2,
                                                           cmap = "rainbow")
fig.show()

## Extra: decoding
[Ref](https://cebra.ai/docs/demo_notebooks/Demo_decoding.html)

In [ ]:
import sklearn.metrics

# Define decoding function with kNN decoder. For a simple demo, we will use the fixed number of neighbors 36.
def decoding_pos_dir(embedding_train, embedding_test, label_train, label_test):
   pos_decoder = cebra.KNNDecoder(n_neighbors=100, metric="cosine")

   pos_decoder.fit(embedding_train, label_train)

   pos_pred = pos_decoder.predict(embedding_test)
   #print(pos_pred)

   prediction = np.stack([pos_pred],axis = 1)
   #print(prediction)
   test_score = sklearn.metrics.r2_score(label_test, pos_pred)
   pos_test_err = np.median(abs(pos_pred[:,0] - label_test[:, 0]))
   pos_test_score = sklearn.metrics.r2_score(label_test[:, 0], pos_pred[:,0])

   return prediction,test_score, pos_test_err, pos_test_score